In [1]:
import pandas as pd
import os

school_csv = os.path.join("Resources", "schools_complete.csv")
student_csv = os.path.join("Resources", "students_complete.csv")

school_data = pd.read_csv(school_csv)
student_data = pd.read_csv(student_csv)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])